## **model  imports and model architecture**

---



In [7]:
import torch
print(torch.__version__)

2.2.0


In [8]:
pip show torch torchvision

Name: torch
Version: 2.2.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /Users/francis/.pyenv/versions/3.12.9/envs/lewagon/lib/python3.12/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: torchvision
---
Name: torchvision
Version: 0.17.0
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /Users/francis/.pyenv/versions/3.12.9/envs/lewagon/lib/python3.12/site-packages
Requires: numpy, pillow, requests, torch
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [9]:
import torchvision
print(torchvision.__version__)

0.17.0


In [10]:
# CELL 1: IMPORTS AND SETUP
# ========================
# Import all necessary libraries and set random seeds for reproducibility

import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import pandas as pd
from tqdm import tqdm
import wandb  # for experiment tracking
import os


# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [11]:
os.environ["WANDB_API_KEY"] = "69077076c3f065786507bee38a48a87779089155"

In [ ]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "EfficientNet-B4_XRay.ipynb"


In [ ]:
# # 1. Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# # 2. Unzip your dataset (adjust the path to your zip file as needed)
# !unzip "/content/drive/My Drive/datasets/xray_images.zip" -d /content/xray_images

In [ ]:
# CELL 2: DATASET EXPLORATION
data_path = '/kaggle/input/your-dataset-name'  # Update with your dataset name
print("Root contents:", os.listdir(data_path))
for folder in os.listdir(data_path):
    print(f"{folder} contents:", os.listdir(os.path.join(data_path, folder)))

In [ ]:
import os
import shutil
import random
from pathlib import Path

# Set random seed for reproducibility
random.seed(42)

# Path to your original dataset (with class folders only)
original_data_dir = Path('/kaggle/input/x-ray-images')  # adjust as needed
split_data_dir = Path('/kaggle/working/xray_split')
split_data_dir.mkdir(parents=True, exist_ok=True)

classes = ['Covid', 'Normal', 'Pneumonia']
split_ratio = 0.8  # 80% train, 20% val

for cls in classes:
    images = list((original_data_dir / cls).glob('*'))
    random.shuffle(images)
    split = int(len(images) * split_ratio)
    train_imgs = images[:split]
    val_imgs = images[split:]

    # Create class folders for train and val
    (split_data_dir / 'train' / cls).mkdir(parents=True, exist_ok=True)
    (split_data_dir / 'val' / cls).mkdir(parents=True, exist_ok=True)

    # Copy images
    for img in train_imgs:
        shutil.copy(img, split_data_dir / 'train' / cls / img.name)
    for img in val_imgs:
        shutil.copy(img, split_data_dir / 'val' / cls / img.name)

In [ ]:
train_dataset = XRayDataset('/kaggle/working/xray_split/train', transform=train_transform)
val_dataset = XRayDataset('/kaggle/working/xray_split/val', transform=val_transform)

In [13]:
# CELL 2: MODEL ARCHITECTURE
# =========================
# EfficientNet-B4 based classifier with custom head for X-ray classification

class XRayClassifier(nn.Module):
    def __init__(self, num_classes=3, pretrained=True):
        super(XRayClassifier, self).__init__()
        # Load pretrained EfficientNet-B4
        self.model = models.efficientnet_b4(pretrained=pretrained)

        # Modify the classifier for our number of classes
        num_features = self.model.classifier[1].in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(p=0.4),
            nn.Linear(num_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# **Dataset and Data Transforms**

In [14]:
# CELL 3: DATASET AND DATA TRANSFORMS
# ==================================
# Custom dataset class and data augmentation transforms

class XRayDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.target_transform = target_transform

        # Find all class directories
        self.class_dirs = [d for d in self.data_dir.iterdir() if d.is_dir()]

        # Create class to index mapping
        self.classes = sorted([d.name for d in self.class_dirs])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

        # Load all image paths and labels
        self.samples = self._load_samples()

    def _load_samples(self):
        samples = []
        for class_dir in self.class_dirs:
            class_name = class_dir.name
            class_idx = self.class_to_idx[class_name]
            image_files = list(class_dir.glob('*.*'))
            for image_path in image_files:
                samples.append((str(image_path), class_idx))
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

# Data transforms with augmentation for training and validation
train_transform = transforms.Compose([
    transforms.Resize((380, 380)),  # EfficientNet-B4 input size
    transforms.RandomCrop((380, 380)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

In [ ]:
# CELL 3.5: DATASET LOADING AND ANALYSIS
# =====================================
# Load and verify datasets
print("Loading datasets...")
train_dataset = XRayDataset('/kaggle/input/x-ray-images/train', transform=train_transform)
val_dataset = XRayDataset('/kaggle/input/x-ray-images/val', transform=val_transform)

model_save_path = '/kaggle/working'

# Print dataset information
print(f"\nDataset Information:")
print(f"Number of classes: {len(train_dataset.classes)}")
print(f"Classes: {train_dataset.classes}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

# Optional: Analyze class distribution
def analyze_class_distribution(dataset):
    class_counts = {}
    for _, label in dataset.samples:
        class_name = dataset.classes[label]
        class_counts[class_name] = class_counts.get(class_name, 0) + 1
    return class_counts

train_distribution = analyze_class_distribution(train_dataset)
val_distribution = analyze_class_distribution(val_dataset)

print("\nClass Distribution:")
print("Training set:")
for cls, count in train_distribution.items():
    print(f"  {cls}: {count} samples")
print("\nValidation set:")
for cls, count in val_distribution.items():
    print(f"  {cls}: {count} samples")

Loading datasets...

Dataset Information:
Number of classes: 3
Classes: [' Pneumonia', 'Covid', 'Normal']
Training samples: 3755
Validation samples: 1473

Class Distribution:
Training set:
   Pneumonia: 1359 samples
  Normal: 1259 samples
  Covid: 1137 samples

Validation set:
  Pneumonia: 441 samples
  Normal: 543 samples
  Covid: 489 samples


In [16]:
# CELL 4: UTILITY FUNCTIONS
# ========================
# Common utility functions for metrics calculation and logging

def calculate_metrics(outputs, targets, criterion):
    """Calculate common metrics for a batch."""
    loss = criterion(outputs, targets)
    _, predicted = outputs.max(1)
    correct = predicted.eq(targets).sum().item()
    total = targets.size(0)
    return loss, correct, total, predicted

def log_metrics(metrics, prefix='', step=None):
    """Log metrics to wandb with optional step."""
    log_dict = {f'{prefix}/{k}': v for k, v in metrics.items()}
    if step is not None:
        log_dict['step'] = step
    wandb.log(log_dict)

def save_checkpoint(model, optimizer, scheduler, epoch, best_acc, path):
    """Save model checkpoint."""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_acc': best_acc,
    }, path)


# **Visualization and Analysis Functions**

In [17]:
# CELL 5: VISUALIZATION FUNCTIONS
# ==============================
# Functions for plotting and analyzing model performance

def plot_training_history(history):
    """Plot training and validation metrics."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot loss
    ax1.plot(history['train_loss'], label='Train Loss')
    ax1.plot(history['val_loss'], label='Val Loss')
    ax1.set_title('Loss Over Time')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Plot accuracy
    ax2.plot(history['train_acc'], label='Train Acc')
    ax2.plot(history['val_acc'], label='Val Acc')
    ax2.set_title('Accuracy Over Time')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()

    plt.tight_layout()
    return fig

def plot_confusion_matrix(y_true, y_pred, classes):
    """Plot confusion matrix."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    return plt.gcf()

def plot_roc_curves(y_true, y_probs, classes):
    """Plot ROC curves for each class."""
    plt.figure(figsize=(10, 8))

    for i, class_name in enumerate(classes):
        fpr, tpr, _ = roc_curve(y_true == i, y_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_name} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves')
    plt.legend(loc="lower right")
    return plt.gcf()

def analyze_predictions(y_true, y_pred, y_probs, classes):
    """Analyze model predictions and generate comprehensive report."""
    # Classification report
    report = classification_report(y_true, y_pred, target_names=classes, output_dict=True)
    report_df = pd.DataFrame(report).transpose()

    # Per-class metrics
    class_metrics = {}
    for i, class_name in enumerate(classes):
        class_metrics[class_name] = {
            'accuracy': report[class_name]['precision'],
            'precision': report[class_name]['precision'],
            'recall': report[class_name]['recall'],
            'f1-score': report[class_name]['f1-score'],
            'support': report[class_name]['support']
        }

    return report_df, class_metrics


# **Training and Validation Functions with Metrics Tracking:**

In [18]:
# CELL 6: TRAINING AND VALIDATION FUNCTIONS
# ========================================
# Core training and validation loops with metrics tracking

def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """Train model for one epoch."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    pbar = tqdm(train_loader, desc=f'Epoch {epoch} [Train]')
    for batch_idx, (data, targets) in enumerate(pbar):
        data, targets = data.to(device), targets.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(data)
        loss, batch_correct, batch_total, predicted = calculate_metrics(outputs, targets, criterion)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update metrics
        running_loss += loss.item()
        correct += batch_correct
        total += batch_total
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

        # Update progress bar
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })

        # Log batch metrics
        log_metrics({
            'batch_loss': loss.item(),
            'batch_acc': 100.*correct/total
        }, prefix='train', step=epoch*len(train_loader) + batch_idx)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc, all_preds, all_labels

def validate_epoch(model, val_loader, criterion, device, epoch):
    """Validate model for one epoch."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        pbar = tqdm(val_loader, desc=f'Epoch {epoch} [Val]')
        for data, targets in pbar:
            data, targets = data.to(device), targets.to(device)

            # Forward pass
            outputs = model(data)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            loss, batch_correct, batch_total, predicted = calculate_metrics(outputs, targets, criterion)

            # Update metrics
            running_loss += loss.item()
            correct += batch_correct
            total += batch_total
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

            # Update progress bar
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc, all_preds, all_labels, all_probs


# **Main Training Loop with Weights & Biases Integration**

In [19]:
# CELL 7: MAIN TRAINING LOOP
# =========================
# Complete training loop with early stopping and model checkpointing

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                num_epochs, device, classes, model_save_path='./models'):
    """Main training loop with early stopping and checkpointing."""
    # Initialize wandb
    wandb.init(project="xray-classification", name="efficientnet-b4")

    # Initialize history
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [],
        'learning_rates': []
    }

    best_acc = 0.0
    patience = 10
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        # Train
        train_loss, train_acc, train_preds, train_labels = train_epoch(
            model, train_loader, criterion, optimizer, device, epoch
        )

        # Validate
        val_loss, val_acc, val_preds, val_labels, val_probs = validate_epoch(
            model, val_loader, criterion, device, epoch
        )

        # Get current learning rate
        current_lr = optimizer.param_groups[0]['lr']

        # Update learning rate
        scheduler.step(val_loss)

        # Store metrics
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['learning_rates'].append(current_lr)

        # Log epoch metrics
        log_metrics({
            'loss': train_loss,
            'acc': train_acc,
            'val_loss': val_loss,
            'val_acc': val_acc,
            'learning_rate': current_lr
        }, step=epoch)

        # Generate and log visualizations
        if epoch % 5 == 0:  # Every 5 epochs
            # Plot training history
            history_fig = plot_training_history(history)
            wandb.log({"training_history": wandb.Image(history_fig)})

            # Plot confusion matrix
            cm_fig = plot_confusion_matrix(val_labels, val_preds, classes)
            wandb.log({"confusion_matrix": wandb.Image(cm_fig)})

            # Plot ROC curves
            roc_fig = plot_roc_curves(val_labels, val_probs, classes)
            wandb.log({"roc_curves": wandb.Image(roc_fig)})

            # Analyze predictions
            report_df, class_metrics = analyze_predictions(
                val_labels, val_preds, val_probs, classes
            )
            wandb.log({"classification_report": wandb.Table(dataframe=report_df)})

        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            save_checkpoint(
                model, optimizer, scheduler, epoch, best_acc,
                f'{model_save_path}/best_model.pth'
            )
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Early stopping
        if epochs_no_improve == patience:
            print(f"Early stopping after {epoch+1} epochs")
            break

    wandb.finish()
    return history


# **Initialization Protocol**

In [20]:
# CELL 8: INITIALIZATION AND TRAINING
# ==================================
# Initialize model and start training

# Initialize model and training components
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = XRayClassifier(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=5, factor=0.5
)

# Train model
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=50,
    device=device,
    classes=train_dataset.classes
)

/Users/francis/.pyenv/versions/3.12.9/envs/lewagon/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/francis/.pyenv/versions/3.12.9/envs/lewagon/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find EfficientNet-B4_XRay.ipynb.
wandb: Currently logged in as: fokanu (fokanu-sage-eye) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 0 [Train]:   0%|          | 0/118 [00:19<?, ?it/s]


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x32fcd4560>> (for post_run_cell), with arguments args (<ExecutionResult object at 32f90ac00, execution_count=20 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 32f90ab70, raw_cell="# CELL 8: INITIALIZATION AND TRAINING
# ==========.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/francis/code/doctai-health-hub/ml_models/xray_classifier/EfficientNet-B4%20XRay.ipynb#X22sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe